In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pulp
from pulp import LpProblem, LpMaximize, LpVariable, lpSum

In [2]:
bulls = pd.read_csv('../data/bulls.csv')
cows = pd.read_csv('../data/cows.csv')
pedigree = pd.read_csv('../data/pedigree.csv')

In [3]:
imputer = KNNImputer(n_neighbors=3)
missing_ebv = imputer.fit_transform(bulls[['ebv', 'descendants_count']])
mask = bulls['ebv'].isna()
bulls.loc[mask, 'ebv'] = missing_ebv[mask.values, 0]

In [4]:
cows['ebv'] = cows['ebv'].fillna(cows['ebv'].median())

In [5]:
cows

,id,ebv
0,RU00000090385,266.5
1,DE00000090396,419.9
2,NL00000090422,-582.8
3,GB00000090437,443.8
4,DE00000090443,173.3
...,...,...
17172,DE00000051612,419.9
17173,FR00000048066,73.4
17174,RU00000048290,137.7
17175,US00000013052,1080.2


In [6]:
pedigree["mother_id"] = pedigree["mother_id"].where(pedigree["mother_id"].notna(), None)
pedigree["father_id"] = pedigree["father_id"].where(pedigree["father_id"].notna(), None)

In [7]:
from functools import lru_cache

# Словари mother и father: id → parent_id или None
mother = dict(zip(pedigree.id, pedigree.mother_id))
father = dict(zip(pedigree.id, pedigree.father_id))

In [8]:
@lru_cache(maxsize=None)
def kin(i: str, j: str) -> float:
    """Коэффициент родства φ(i,j)."""
    if i is None or j is None:
        return 0.0
    if i == j:
        # Пример: считаем, что в файле нет инбридинга, так что φ(ii)=0.5
        return 0.5
    mj = mother.get(j)
    fj = father.get(j)
    return 0.5 * (kin(i, mj) + kin(i, fj))

In [9]:
MAX_KINSHIP = 0.05
MAX_BULL_USAGE = int(len(cows) * 0.1)

# Предварительно создаём списки EBV
# Для быков
bulls['ebv'] = pd.to_numeric(bulls['ebv'], errors='coerce')  # нечисловые → NaN
bulls = bulls.dropna(subset=['ebv']).reset_index(drop=True)

In [10]:
bulls

,id,descendants_count,ebv
0,FR00000000479,476,690.700000
1,NL00000001842,325,-597.600000
2,US00000003013,436,1528.600000
3,US00000001653,1097,1193.200000
4,DE00000001742,496,1510.600000
5,GB00000002348,522,1315.300000
6,US00000002804,355,1444.300000
7,FR00000002912,1250,1238.700000
8,DE00000003760,369,1138.500000
9,DE00000003486,472,1085.500000


In [11]:
# Для коров
cows['ebv'] = pd.to_numeric(cows['ebv'], errors='coerce')


In [12]:
cows = cows.dropna(subset=['ebv']).reset_index(drop=True)
# При этом словари EBV:
bull_ebv = bulls.set_index("id")["ebv"].to_dict()
cow_ebv  = cows.set_index("id")["ebv"].to_dict()

In [13]:
# Собираем список допустимых пар (cow_id, bull_id, expected_ebv)
from math import isfinite

valid_pairs = []
for cow_id in cows["id"]:
    for bull_id in bulls["id"]:
        # Родственное ограничение
        if kin(cow_id, bull_id) <= MAX_KINSHIP:
            e = (cow_ebv[cow_id] + bull_ebv[bull_id]) / 2
            # Добавляем только конечные, не NaN и не inf
            if isfinite(e):
                valid_pairs.append((cow_id, bull_id, e))

In [14]:
valid_pairs

[('RU00000090385', 'FR00000000479', 478.6),
 ('RU00000090385', 'NL00000001842', -165.55),
 ('RU00000090385', 'US00000003013', 897.55),
 ('RU00000090385', 'US00000001653', 729.85),
 ('RU00000090385', 'DE00000001742', 888.55),
 ('RU00000090385', 'GB00000002348', 790.9),
 ('RU00000090385', 'US00000002804', 855.4),
 ('RU00000090385', 'FR00000002912', 752.6),
 ('RU00000090385', 'DE00000003760', 702.5),
 ('RU00000090385', 'DE00000003486', 676.0),
 ('RU00000090385', 'FR00000003214', -104.5),
 ('RU00000090385', 'RU00000003147', 545.7),
 ('RU00000090385', 'RU00000002534', 370.95),
 ('RU00000090385', 'NL00000003091', 402.75),
 ('RU00000090385', 'GB00000002585', 720.2),
 ('RU00000090385', 'GB00000002899', 400.7),
 ('RU00000090385', 'RU00000002787', 126.75),
 ('RU00000090385', 'US00000003507', 1024.7),
 ('RU00000090385', 'US00000000795', 1067.2),
 ('RU00000090385', 'US00000003459', 886.55),
 ('RU00000090385', 'NL00000051437', 519.5),
 ('RU00000090385', 'FR00000051194', 306.55),
 ('RU00000090385', 

In [15]:
model1 = pulp.LpProblem("Maximize_EBV", pulp.LpMaximize)
x1 = {(c,b): pulp.LpVariable(f"x1_{c}_{b}", cat="Binary") for c,b,_ in valid_pairs}

# Цель
model1 += pulp.lpSum(x1[(c,b)] * e for c,b,e in valid_pairs)

# Каждая корова — ровно одна пара
for c in cows["id"]:
    model1 += pulp.lpSum(x1[(c,b)] for _,b,e in valid_pairs if _==c) == 1

# Не более 10% коров на быка
for b in bulls["id"]:
    model1 += pulp.lpSum(x1[(c,b)] for c,_,e in valid_pairs if _==b) <= MAX_BULL_USAGE

model1.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=1800))
opt_total_ebv = pulp.value(model1.objective)

In [ ]:
model2 = pulp.LpProblem("Maximize_Variance", pulp.LpMaximize)
x2 = {(c,b): pulp.LpVariable(f"x2_{c}_{b}", cat="Binary") for c,b,_ in valid_pairs}

# Жёстко фиксируем общий EBV
model2 += pulp.lpSum(x2[(c,b)] * e for c,b,e in valid_pairs) == opt_total_ebv

# Целевая — сумма квадратов
model2 += pulp.lpSum(x2[(c,b)] * (e**2) for c,b,e in valid_pairs)

# Те же ограничения
for c in cows["id"]:
    model2 += pulp.lpSum(x2[(c,b)] for _,b,e in valid_pairs if _==c) == 1
for b in bulls["id"]:
    model2 += pulp.lpSum(x2[(c,b)] for c,_,e in valid_pairs if _==b) <= MAX_BULL_USAGE

model2.solve(pulp.PULP_CBC_CMD(msg=False, timeLimit=1800))

0

In [20]:
assignments = [
    {"cow_id": c, "bull_id": b}
    for (c,b,e) in valid_pairs
    if pulp.value(x2[(c,b)]) > 0.5
]
out = pd.DataFrame(assignments)

In [21]:
out

,cow_id,bull_id
0,RU00000090385,GB00000002348
1,DE00000090396,US00000003459
2,NL00000090422,GB00000002585
3,GB00000090437,US00000003459
4,DE00000090443,FR00000002912
...,...,...
17172,DE00000051612,US00000003459
17173,FR00000048066,FR00000002912
17174,RU00000048290,FR00000002912
17175,US00000013052,US00000000795


In [ ]:
out.to_csv("cow_bull_assignments.csv")